In [ ]:
import Merge_Cartesian 

from decimal import *
import math
import copy
import numpy as np
import matplotlib.cm as cm
import matplotlib.pyplot as plt
from matplotlib.animation import FFMpegWriter

In [ ]:
plt.rcParams['animation.ffmpeg_path'] = 'C:\\Users\\Jonathan\\Documents\\Academic\\Masters\\Simulator\\Git\\Main_Algorithm\\ffmpeg\\bin\\ffmpeg.exe'

import ipywidgets as widgets
from IPython.display import display

# Get Raw Wavefront Arrays

In [ ]:
%matplotlib widget
L_Time = '1'
C_Time = '9.9'

L_impedance = '1000'
C_impedance = '1'

number_periods = '1'

#  INDUCTANCE, CAPACITANCE, LENGTH
Inductor_List = [L_impedance,L_Time,'1']
Capacitor_List = [C_impedance,C_Time,'1']

# VOLTAGE, PERIODS, BUCK, LOAD
Circuit_List = ['1',number_periods,False,'12']

In [ ]:
(data_input,
data_output
) = Merge_Cartesian.Process_Wavefronts(Inductor_List,Capacitor_List, Circuit_List)

In [ ]:
fig_initial_colour, ax_initial_colour = plt.subplot_mosaic([['A','B'],['C','D']])

In [ ]:
Merge_Cartesian.plot_fanout_seismic(data_output.Voltage_Interconnect_Inductor,ax_initial_colour['A'],"Inductor Voltage Fanout",True,True)
Merge_Cartesian.plot_fanout_seismic(data_output.Voltage_Interconnect_Capacitor,ax_initial_colour['B'],"Capacitor Voltage Fanout",True)
Merge_Cartesian.plot_fanout_seismic(data_output.Current_Interconnect_Inductor,ax_initial_colour['C'],"Inductor Current Fanout",True)
Merge_Cartesian.plot_fanout_seismic(data_output.Current_Interconnect_Capacitor,ax_initial_colour['D'],"Capacitor Current Fanout",True)

In [ ]:
fig_time, ax_time = plt.subplots()

Merge_Cartesian.plot_fanout_colour(data_output.Time,ax_time,"Time Fanout Plot")

fig, ax = plt.subplots(subplot_kw={"projection": "3d"})

Nx,Ny = Voltage_Capacitor.shape
x = np.linspace(0, 1, Nx)
y = np.linspace(0, 1, Ny)
X, Y = np.meshgrid(x, y)

ax.plot_surface(X,Y,Voltage_Capacitor,cmap=cm.coolwarm)

# Trying to understand Fanout Frequency

In [ ]:
fig_cross, ax_cross = plt.subplots(1,3)

In [ ]:
Merge_Cartesian.plot_fanout_crossection(Merge_Cartesian.get_voltage_array (data_output.Wavefronts_Sending_Capacitor),ax_cross,22,"Capacitor Sending Voltage",False)

# Higher Order Merging
-----

## Visualising merigng steps iteritevly

In [ ]:
a = data_input.a
b = data_input.b

if(data_input.is_Higher_Merging):

    # Frist Row
    fig_merge,ax_merge = plt.subplots(3,3)
    Merge_Cartesian.plot_fanout_seismic(data_output.Voltage_Interconnect_Capacitor, ax_merge[0,0],"Input Array",False)

    voltage_capacitor_merged = Merge_Cartesian.multiplicative_merge_cycle(data_output.Voltage_Interconnect_Capacitor,a,b)
    Merge_Cartesian.plot_fanout_seismic(voltage_capacitor_merged, ax_merge[0,1],"1",False)

    voltage_capacitor_merged = Merge_Cartesian.multiplicative_merge_cycle(voltage_capacitor_merged,a,b)
    Merge_Cartesian.plot_fanout_seismic(voltage_capacitor_merged, ax_merge[0,2],"2",False)


    # Second Row
    voltage_capacitor_merged = Merge_Cartesian.multiplicative_merge_cycle(voltage_capacitor_merged,a,b)
    Merge_Cartesian.plot_fanout_seismic(voltage_capacitor_merged, ax_merge[1,0],"3",False)

    voltage_capacitor_merged = Merge_Cartesian.multiplicative_merge_cycle(voltage_capacitor_merged,a,b)
    Merge_Cartesian.plot_fanout_seismic(voltage_capacitor_merged, ax_merge[1,1],"4",False)

    voltage_capacitor_merged = Merge_Cartesian.multiplicative_merge_cycle(voltage_capacitor_merged,a,b)
    Merge_Cartesian.plot_fanout_seismic(voltage_capacitor_merged, ax_merge[1,2],"5",False)


    # Third Row
    voltage_capacitor_merged = Merge_Cartesian.multiplicative_merge_cycle(voltage_capacitor_merged,a,b)
    Merge_Cartesian.plot_fanout_seismic(voltage_capacitor_merged, ax_merge[2,0],"6",False)

    voltage_capacitor_merged = Merge_Cartesian.multiplicative_merge_cycle(voltage_capacitor_merged,a,b)
    Merge_Cartesian.plot_fanout_seismic(voltage_capacitor_merged, ax_merge[2,1],"7",False)

    voltage_capacitor_merged = Merge_Cartesian.multiplicative_merge_cycle(voltage_capacitor_merged,a,b)
    Merge_Cartesian.plot_fanout_seismic(voltage_capacitor_merged, ax_merge[2,2],"8",False)


## Doing full Merging Cycle

In [ ]:
data_output_merged = Merge_Cartesian.Higher_Order_Merging(data_input,data_output)

In [ ]:
fig_merged, ax_merged = plt.subplot_mosaic([['A','B'],['C','D']])

Merge_Cartesian.plot_fanout_seismic(data_output_merged.Voltage_Interconnect_Inductor,ax_merged['A'],"Merged Inductor Voltage",True,True)
Merge_Cartesian.plot_fanout_seismic(data_output_merged.Current_Interconnect_Inductor,ax_merged['C'],"Merged Inductor Current",True)

Merge_Cartesian.plot_fanout_seismic(data_output_merged.Voltage_Interconnect_Capacitor,ax_merged['B'],"Merged Capacitor Voltage",True)
Merge_Cartesian.plot_fanout_seismic(data_output_merged.Current_Interconnect_Capacitor,ax_merged['D'],"Merged Capacitor Current",True)


## Order merged output

In [ ]:
data_output_ordered = Merge_Cartesian.Order_Data_Output_Merged(data_input,data_output_merged)

In [ ]:
fig_series_voltage_2,ax_series_voltage_2 =plt.subplot_mosaic([['A','C'],['B','C']])

Merge_Cartesian.plot_time_interconnect_3(data_output_merged,data_output_ordered,ax_series_voltage_2,'current capacitor')

# Wavefront Positions at Time

In [ ]:
Merge_Cartesian.spatial_investigator_ui(data_input, data_output_merged, data_output_ordered)

In [ ]:
Merge_Cartesian.video_save_ui(data_input, data_output_merged, data_output_ordered)

## Gradient Follow

In [ ]:
A = Decimal(L_Time)
B = Decimal(C_Time)

LCM , GCF = Merge_Cartesian.lcm_gcd(A,B)

a = int((B)/(GCF))
b = int((A)/(GCF))

A_to_B_x = 0
A_to_B_y = 0

for  y in range(1,data_input.Number_of_Wavefronts):
    
    if( (y*B - GCF )%A == 0):
        A_to_B_x = (y*B - GCF)/A
        A_to_B_y = y
        break
    
    if(y == data_input.Number_of_Wavefronts -1):
        print('Eish')
        

B_to_A_x = 0
B_to_A_y = 0

for  y in range(1,data_input.Number_of_Wavefronts):
    
    if( (y*B + GCF )%A == 0):
        B_to_A_x = (y*B + GCF)/A
        B_to_A_y = y
        break
    
    if(y == data_input.Number_of_Wavefronts -1):
        print('Eish')

print('A :',A)
print('B :',B)
print('GCF :',GCF)
print('LCM :',LCM)
print('a :',a)
print('b :',b)
print('A_to_B_x : ' + str(A_to_B_x))
print('A_to_B_y : ' + str(A_to_B_y))
print('B_to_A_x : ' + str(B_to_A_x))
print('B_to_A_y : ' + str(B_to_A_y))

In [ ]:
data_input.Number_of_Wavefronts